# Fetch historical data

Python script to fetch historical data from binance using ccxt

In [1]:
# Install openpyxl and CCXT
# !pip install openpyxl ccxt

In [2]:
import os
from pathlib import Path

import sys
import csv

# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

import ccxt


# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)


In [9]:
scrape_candles_to_csv('btc_usdt_1m.csv', 'binance', 3, 'BTC/USDT', '1m', '2017-01-0100:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/eth_btc_1m.csv', 'binance', 3, 'ETH/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/ltc_btc_1m.csv', 'binance', 3, 'LTC/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/xlm_btc_1m.csv', 'binance', 3, 'XLM/BTC', '1m', '2018-01-01T00:00:00Z', 1000)

1000 BTC/USDT candles in total from 2019-10-27T11:51:00.000Z to 2019-10-28T04:30:00.000Z
2000 BTC/USDT candles in total from 2019-10-26T19:11:00.000Z to 2019-10-28T04:30:00.000Z
3000 BTC/USDT candles in total from 2019-10-26T02:31:00.000Z to 2019-10-28T04:30:00.000Z
4000 BTC/USDT candles in total from 2019-10-25T09:51:00.000Z to 2019-10-28T04:30:00.000Z
5000 BTC/USDT candles in total from 2019-10-24T17:11:00.000Z to 2019-10-28T04:30:00.000Z
6000 BTC/USDT candles in total from 2019-10-24T00:31:00.000Z to 2019-10-28T04:30:00.000Z
7000 BTC/USDT candles in total from 2019-10-23T07:51:00.000Z to 2019-10-28T04:30:00.000Z
8000 BTC/USDT candles in total from 2019-10-22T15:11:00.000Z to 2019-10-28T04:30:00.000Z
9000 BTC/USDT candles in total from 2019-10-21T22:31:00.000Z to 2019-10-28T04:30:00.000Z
10000 BTC/USDT candles in total from 2019-10-21T05:51:00.000Z to 2019-10-28T04:30:00.000Z
11000 BTC/USDT candles in total from 2019-10-20T13:11:00.000Z to 2019-10-28T04:30:00.000Z
12000 BTC/USDT cand

93000 BTC/USDT candles in total from 2019-08-24T14:31:00.000Z to 2019-10-28T04:30:00.000Z
94000 BTC/USDT candles in total from 2019-08-23T21:51:00.000Z to 2019-10-28T04:30:00.000Z
95000 BTC/USDT candles in total from 2019-08-23T05:11:00.000Z to 2019-10-28T04:30:00.000Z
96000 BTC/USDT candles in total from 2019-08-22T12:31:00.000Z to 2019-10-28T04:30:00.000Z
97000 BTC/USDT candles in total from 2019-08-21T19:51:00.000Z to 2019-10-28T04:30:00.000Z
98000 BTC/USDT candles in total from 2019-08-21T03:11:00.000Z to 2019-10-28T04:30:00.000Z
99000 BTC/USDT candles in total from 2019-08-20T10:31:00.000Z to 2019-10-28T04:30:00.000Z
100000 BTC/USDT candles in total from 2019-08-19T17:51:00.000Z to 2019-10-28T04:30:00.000Z
101000 BTC/USDT candles in total from 2019-08-19T01:11:00.000Z to 2019-10-28T04:30:00.000Z
102000 BTC/USDT candles in total from 2019-08-18T08:31:00.000Z to 2019-10-28T04:30:00.000Z
103000 BTC/USDT candles in total from 2019-08-17T15:51:00.000Z to 2019-10-28T04:30:00.000Z
104000

184000 BTC/USDT candles in total from 2019-06-22T09:51:00.000Z to 2019-10-28T04:30:00.000Z
185000 BTC/USDT candles in total from 2019-06-21T17:11:00.000Z to 2019-10-28T04:30:00.000Z
186000 BTC/USDT candles in total from 2019-06-21T00:31:00.000Z to 2019-10-28T04:30:00.000Z
187000 BTC/USDT candles in total from 2019-06-20T07:51:00.000Z to 2019-10-28T04:30:00.000Z
188000 BTC/USDT candles in total from 2019-06-19T15:11:00.000Z to 2019-10-28T04:30:00.000Z
189000 BTC/USDT candles in total from 2019-06-18T22:31:00.000Z to 2019-10-28T04:30:00.000Z
190000 BTC/USDT candles in total from 2019-06-18T05:51:00.000Z to 2019-10-28T04:30:00.000Z
191000 BTC/USDT candles in total from 2019-06-17T13:11:00.000Z to 2019-10-28T04:30:00.000Z
192000 BTC/USDT candles in total from 2019-06-16T20:31:00.000Z to 2019-10-28T04:30:00.000Z
193000 BTC/USDT candles in total from 2019-06-16T03:51:00.000Z to 2019-10-28T04:30:00.000Z
194000 BTC/USDT candles in total from 2019-06-15T11:11:00.000Z to 2019-10-28T04:30:00.000Z

275000 BTC/USDT candles in total from 2019-04-20T13:00:00.000Z to 2019-10-28T04:30:00.000Z
276000 BTC/USDT candles in total from 2019-04-19T20:20:00.000Z to 2019-10-28T04:30:00.000Z
277000 BTC/USDT candles in total from 2019-04-19T03:40:00.000Z to 2019-10-28T04:30:00.000Z
278000 BTC/USDT candles in total from 2019-04-18T11:00:00.000Z to 2019-10-28T04:30:00.000Z
279000 BTC/USDT candles in total from 2019-04-17T18:20:00.000Z to 2019-10-28T04:30:00.000Z
280000 BTC/USDT candles in total from 2019-04-17T01:40:00.000Z to 2019-10-28T04:30:00.000Z
281000 BTC/USDT candles in total from 2019-04-16T09:00:00.000Z to 2019-10-28T04:30:00.000Z
282000 BTC/USDT candles in total from 2019-04-15T16:20:00.000Z to 2019-10-28T04:30:00.000Z
283000 BTC/USDT candles in total from 2019-04-14T23:40:00.000Z to 2019-10-28T04:30:00.000Z
284000 BTC/USDT candles in total from 2019-04-14T07:00:00.000Z to 2019-10-28T04:30:00.000Z
285000 BTC/USDT candles in total from 2019-04-13T14:20:00.000Z to 2019-10-28T04:30:00.000Z

366000 BTC/USDT candles in total from 2019-02-16T08:20:00.000Z to 2019-10-28T04:30:00.000Z
367000 BTC/USDT candles in total from 2019-02-15T15:40:00.000Z to 2019-10-28T04:30:00.000Z
368000 BTC/USDT candles in total from 2019-02-14T23:00:00.000Z to 2019-10-28T04:30:00.000Z
369000 BTC/USDT candles in total from 2019-02-14T06:20:00.000Z to 2019-10-28T04:30:00.000Z
370000 BTC/USDT candles in total from 2019-02-13T13:40:00.000Z to 2019-10-28T04:30:00.000Z
371000 BTC/USDT candles in total from 2019-02-12T21:00:00.000Z to 2019-10-28T04:30:00.000Z
372000 BTC/USDT candles in total from 2019-02-12T04:20:00.000Z to 2019-10-28T04:30:00.000Z
373000 BTC/USDT candles in total from 2019-02-11T11:40:00.000Z to 2019-10-28T04:30:00.000Z
374000 BTC/USDT candles in total from 2019-02-10T19:00:00.000Z to 2019-10-28T04:30:00.000Z
375000 BTC/USDT candles in total from 2019-02-10T02:20:00.000Z to 2019-10-28T04:30:00.000Z
376000 BTC/USDT candles in total from 2019-02-09T09:40:00.000Z to 2019-10-28T04:30:00.000Z

457000 BTC/USDT candles in total from 2018-12-15T03:40:00.000Z to 2019-10-28T04:30:00.000Z
458000 BTC/USDT candles in total from 2018-12-14T11:00:00.000Z to 2019-10-28T04:30:00.000Z
459000 BTC/USDT candles in total from 2018-12-13T18:20:00.000Z to 2019-10-28T04:30:00.000Z
460000 BTC/USDT candles in total from 2018-12-13T01:40:00.000Z to 2019-10-28T04:30:00.000Z
461000 BTC/USDT candles in total from 2018-12-12T09:00:00.000Z to 2019-10-28T04:30:00.000Z
462000 BTC/USDT candles in total from 2018-12-11T16:20:00.000Z to 2019-10-28T04:30:00.000Z
463000 BTC/USDT candles in total from 2018-12-10T23:40:00.000Z to 2019-10-28T04:30:00.000Z
464000 BTC/USDT candles in total from 2018-12-10T07:00:00.000Z to 2019-10-28T04:30:00.000Z
465000 BTC/USDT candles in total from 2018-12-09T14:20:00.000Z to 2019-10-28T04:30:00.000Z
466000 BTC/USDT candles in total from 2018-12-08T21:40:00.000Z to 2019-10-28T04:30:00.000Z
467000 BTC/USDT candles in total from 2018-12-08T05:00:00.000Z to 2019-10-28T04:30:00.000Z

548000 BTC/USDT candles in total from 2018-10-12T23:00:00.000Z to 2019-10-28T04:30:00.000Z
549000 BTC/USDT candles in total from 2018-10-12T06:20:00.000Z to 2019-10-28T04:30:00.000Z
550000 BTC/USDT candles in total from 2018-10-11T13:40:00.000Z to 2019-10-28T04:30:00.000Z
551000 BTC/USDT candles in total from 2018-10-10T21:00:00.000Z to 2019-10-28T04:30:00.000Z
552000 BTC/USDT candles in total from 2018-10-10T04:20:00.000Z to 2019-10-28T04:30:00.000Z
553000 BTC/USDT candles in total from 2018-10-09T11:40:00.000Z to 2019-10-28T04:30:00.000Z
554000 BTC/USDT candles in total from 2018-10-08T19:00:00.000Z to 2019-10-28T04:30:00.000Z
555000 BTC/USDT candles in total from 2018-10-08T02:20:00.000Z to 2019-10-28T04:30:00.000Z
556000 BTC/USDT candles in total from 2018-10-07T09:40:00.000Z to 2019-10-28T04:30:00.000Z
557000 BTC/USDT candles in total from 2018-10-06T17:00:00.000Z to 2019-10-28T04:30:00.000Z
558000 BTC/USDT candles in total from 2018-10-06T00:20:00.000Z to 2019-10-28T04:30:00.000Z

639000 BTC/USDT candles in total from 2018-08-10T18:20:00.000Z to 2019-10-28T04:30:00.000Z
640000 BTC/USDT candles in total from 2018-08-10T01:40:00.000Z to 2019-10-28T04:30:00.000Z
641000 BTC/USDT candles in total from 2018-08-09T09:00:00.000Z to 2019-10-28T04:30:00.000Z
642000 BTC/USDT candles in total from 2018-08-08T16:20:00.000Z to 2019-10-28T04:30:00.000Z
643000 BTC/USDT candles in total from 2018-08-07T23:40:00.000Z to 2019-10-28T04:30:00.000Z
644000 BTC/USDT candles in total from 2018-08-07T07:00:00.000Z to 2019-10-28T04:30:00.000Z
645000 BTC/USDT candles in total from 2018-08-06T14:20:00.000Z to 2019-10-28T04:30:00.000Z
646000 BTC/USDT candles in total from 2018-08-05T21:40:00.000Z to 2019-10-28T04:30:00.000Z
647000 BTC/USDT candles in total from 2018-08-05T05:00:00.000Z to 2019-10-28T04:30:00.000Z
648000 BTC/USDT candles in total from 2018-08-04T12:20:00.000Z to 2019-10-28T04:30:00.000Z
649000 BTC/USDT candles in total from 2018-08-03T19:40:00.000Z to 2019-10-28T04:30:00.000Z

730000 BTC/USDT candles in total from 2018-06-08T15:20:00.000Z to 2019-10-28T04:30:00.000Z
731000 BTC/USDT candles in total from 2018-06-07T22:40:00.000Z to 2019-10-28T04:30:00.000Z
732000 BTC/USDT candles in total from 2018-06-07T06:00:00.000Z to 2019-10-28T04:30:00.000Z
733000 BTC/USDT candles in total from 2018-06-06T13:20:00.000Z to 2019-10-28T04:30:00.000Z
734000 BTC/USDT candles in total from 2018-06-05T20:40:00.000Z to 2019-10-28T04:30:00.000Z
735000 BTC/USDT candles in total from 2018-06-05T04:00:00.000Z to 2019-10-28T04:30:00.000Z
736000 BTC/USDT candles in total from 2018-06-04T11:20:00.000Z to 2019-10-28T04:30:00.000Z
737000 BTC/USDT candles in total from 2018-06-03T18:40:00.000Z to 2019-10-28T04:30:00.000Z
738000 BTC/USDT candles in total from 2018-06-03T02:00:00.000Z to 2019-10-28T04:30:00.000Z
739000 BTC/USDT candles in total from 2018-06-02T09:20:00.000Z to 2019-10-28T04:30:00.000Z
740000 BTC/USDT candles in total from 2018-06-01T16:40:00.000Z to 2019-10-28T04:30:00.000Z

821000 BTC/USDT candles in total from 2018-04-06T10:40:00.000Z to 2019-10-28T04:30:00.000Z
822000 BTC/USDT candles in total from 2018-04-05T18:00:00.000Z to 2019-10-28T04:30:00.000Z
823000 BTC/USDT candles in total from 2018-04-05T01:20:00.000Z to 2019-10-28T04:30:00.000Z
824000 BTC/USDT candles in total from 2018-04-04T08:40:00.000Z to 2019-10-28T04:30:00.000Z
825000 BTC/USDT candles in total from 2018-04-03T16:00:00.000Z to 2019-10-28T04:30:00.000Z
826000 BTC/USDT candles in total from 2018-04-02T23:20:00.000Z to 2019-10-28T04:30:00.000Z
827000 BTC/USDT candles in total from 2018-04-02T06:40:00.000Z to 2019-10-28T04:30:00.000Z
828000 BTC/USDT candles in total from 2018-04-01T14:00:00.000Z to 2019-10-28T04:30:00.000Z
829000 BTC/USDT candles in total from 2018-03-31T21:20:00.000Z to 2019-10-28T04:30:00.000Z
830000 BTC/USDT candles in total from 2018-03-31T04:40:00.000Z to 2019-10-28T04:30:00.000Z
831000 BTC/USDT candles in total from 2018-03-30T12:00:00.000Z to 2019-10-28T04:30:00.000Z